# 目标

    1、加载昨天fine-tune过的googlenet。
    2、并使用它进行预测

In [1]:
import tensorflow as tf
from tensorflow.python.platform import gfile
import os
import numpy as np

/Users/roland/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
'''
定义一些全局变量
'''
DATA_DIR='./data/flower_photos/'
INCEPTION_MODEL_DIR = './model/inceptionv3/'
INCEPTION_MODEL_FILE = 'tensorflow_inception_graph.pb'
FC_MODEL_DIR ='./model/fine_tune_fc/'
n_classes = 5

BOTTLENECK_TENSOR_NAME = 'pool_3/_reshape:0' # 瓶颈层输出的张量
JPEG_DATA_TENSOR_NAME = 'DecodeJpeg/contents:0' # 输入张量名字

TEST_SUMMRAY_DIR='./model/logs/'

In [3]:
'''
加载测试文件以及对应的label
'''
def get_test_data():
    # 加载label
    label_dict={}
    with open(os.path.join(DATA_DIR,'labels.txt'),'r') as f:
        for line in f.readlines():
            line=line[1:-1]
            value = line.split('->')[0]
            key = line.split('->')[1].strip()
            label_dict[key]=int(value)
        
    # 拿到所有的文件信息和对应的label
    data_list=[]
    label_list=[]
    for dir_name in os.listdir(DATA_DIR):
        if dir_name in label_dict.keys():
            label_index = label_dict[dir_name]
            for img_name in os.listdir(os.path.join(DATA_DIR,dir_name)):
                abs_path = os.path.join(DATA_DIR,dir_name,img_name)
                img_data = gfile.FastGFile(abs_path,'rb').read()
                
                data_list.append(img_data)
                
                img_label = np.zeros(n_classes,dtype=np.float32)
                img_label[label_index]=1.0
                label_list.append(img_label)
                
    return np.asarray(data_list),np.asarray(label_list)


data_list,label_list=get_test_data()

In [6]:
'''
加载模型
'''
tf.reset_default_graph()
with tf.Graph().as_default() as graph:
#     with gfile.FastGFile(os.path.join(INCEPTION_MODEL_DIR,INCEPTION_MODEL_FILE),'rb') as f:
#         graph_def = tf.GraphDef()
#         graph_def.ParseFromString(f.read())
#         # 加载inception模型，并返回输入张量和输出张量
        
#         bottleneck_tensor,jped_data_tensor = tf.import_graph_def(graph_def,
#                                                                 return_elements=[BOTTLENECK_TENSOR_NAME,
#                                                                                  JPEG_DATA_TENSOR_NAME])
    with tf.Session(graph=graph) as sess:
        # 加载自己写的FC模型
        
        # 由于昨天保存的时候，inception模型一起保存了
        # 所以就是会一起展示
        new_saver = tf.train.import_meta_graph(os.path.join(FC_MODEL_DIR,'model-900.meta'))
        new_saver.restore(sess,tf.train.latest_checkpoint(FC_MODEL_DIR))
        
        
        
        tf.summary.FileWriter(TEST_SUMMRAY_DIR,sess.graph)
        # inception的输入输出
        inc_input = graph.get_tensor_by_name('import/DecodeJpeg/contents:0')
        inc_output = graph.get_tensor_by_name('import/pool_3/_reshape:0')
        
        # 拿到FC模型的输入输出
        fc_input = graph.get_tensor_by_name('BottleneckInputPlaceholder:0')
        fc_output = graph.get_tensor_by_name('final_training_ops/add:0')
        
        
        img_feature_vector  = sess.run(inc_output,feed_dict={inc_input:data_list[5]})
        
        cate_result  = sess.run(fc_output,feed_dict={fc_input:img_feature_vector})
        print(sess.run(tf.argmax(cate_result,1)))

INFO:tensorflow:Restoring parameters from /Users/roland/Desktop/机器学习/01_MyLab/01_tensorflow_test/Tensorflow-Practicing/Day3_GoogleNetTransformerLearning/runs/1549032598/checkpoints/model-900
[1]
